In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
class MLPClassifier_pytorch:
    
    
    def __init__(self, input_dim):
        self.input_dim = input_dim
        self.model = self.MLP(self.input_dim)

    import torch.nn.functional as F
    
    class MLP(torch.nn.Module):
        
        def __init__(self, input_dim, output_dim=11):
            super().__init__()

            self.input_fc = torch.nn.Linear(input_dim, 1024)
            self.output_fc = torch.nn.Linear(1024, output_dim)

        def forward(self, x):

            # x = [batch size, height, width]

            batch_size = x.shape[0]
            # feat_dim = x.shape[-1]
            
            x = x.view(batch_size,-1)

            # x = [batch size, height * width]

            h_1 = torch.nn.functional.relu(self.input_fc(x))

            y_pred = self.output_fc(h_1)

            y_pred = torch.nn.functional.softmax(y_pred, dim=1)
            # y_pred = [batch size, output dim]

            return y_pred
    
    def calculate_accuracy(self, y_pred, y):

        top_pred = y_pred.argmax(1, keepdim=True)
        correct = top_pred.eq(y.view_as(top_pred)).sum()
        acc = correct.float() / y.shape[0]

        return acc.numpy()


    def train(self, x, y, model, optimizer, criterion, device):


        x_ = torch.from_numpy(x).to(device).float()
        import torch.nn.functional as f
        y_ = torch.from_numpy(y).type(torch.LongTensor).to(device).float()
        
        epoch_loss = 0
        epoch_acc = 0

        model.train()

        optimizer.zero_grad()

        y_pred = model(x_)
       
        y_ = y_.type(torch.LongTensor)
        
        # y_special = F.one_hot(y_, num_classes=11)
        loss = criterion(y_pred, y_)

        acc = self.calculate_accuracy(y_pred, y_)

        loss.backward()

        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()

        return 1
    
    
    def fit(self, X_train, Y_train):
        device = torch.device('cpu')

        self.model = self.model.to(device)
        
        num_epochs = 10
        
        criterion = nn.CrossEntropyLoss().to(device)
        optimizer = optim.Adam(self.model.parameters(), lr=0.0001)

        for n in range(num_epochs):
            self.train(X_train, Y_train, self.model, optimizer, criterion, device)
        
        return self
        
    def predict(self, x):
        device = torch.device('cpu')
        x_ = torch.from_numpy(x).to(device).float()
        y_ = self.model(x_)
        y_ret = y_.detach().cpu().numpy()
        
        y_ret = np.argmax(y_ret, axis=1)
        
        return y_ret

In [26]:
def do_everything(selection, ooi_data):

    # selection = ['scr', 'mr']
    # selection = ['lr']
    # selection = ['scr']
    import pickle as pkl

    import io
    import torch

    all_labels = []

    for d in ooi_data:
        temp_labels = []
        
        for r in d['relations']:
            rels = r[1]
            
            # for k in selection:
            #     temp_labels+=rels[k]
            
            temp_labels+=rels['scr']
            temp_labels+=rels['lr']
            temp_labels+=rels['mr']
            
        all_labels+= temp_labels
            
    unique_labels = list(set(all_labels))
    labels_enumeration = {}

    for i, u in enumerate(unique_labels):
        
        if u == '':
            continue
        labels_enumeration[u] = i-1

    # print(labels_enumeration)
    all_videos = []
    vid_to_class = {}

    for d in ooi_data:
        all_videos.append( d['metadata']['yt_id'] )
        vid_to_class[ d['metadata']['yt_id'] ] = d['metadata']['activity name']

    unique_videos = list(set(all_videos))
    videos_enumeration = {}

    for i, u in enumerate(unique_videos):
        
        videos_enumeration[u] = i

    # print(videos_enumeration)
    # print(vid_to_class)
    activity_enumeration = {'AssembleCabinet': 0, 'ChangeCarTire': 1, 'FuelCar': 2, 'InstallBicycleRack': 3,
    'InstallShowerHead': 4, 'ParkParallel': 5, 'ReplaceDoorKnob': 6, 'ReplaceToiletSeat': 7,
    'UseJack': 8, 'PolishCar': 9, 'ReplaceBatteryOnTVControl': 10}
    import numpy as np

    def make_hoi_features(ooi_data, videos_enumeration, labels_enumeration, activity_enumeration,  vid_to_class_map
                        , hoi_ooi_both='hoi'):

        num_rows = len( list(videos_enumeration.keys()) )
        num_cols = len( list(labels_enumeration.keys()) )
        
        X = np.zeros((num_rows, num_cols))
        Y = np.zeros(num_rows)
        num_gifs_for_each_vid = np.zeros(num_rows)
        
        num_gifs = len(ooi_data)
        
        for i in range(num_gifs):

            curr_vid = ooi_data[i]['metadata']['yt_id']
            curr_activity = vid_to_class_map[curr_vid]
            curr_activity_enumeration = activity_enumeration[curr_activity]

            curr_row = videos_enumeration[curr_vid]
            
            
            Y[curr_row] = curr_activity_enumeration
            
            
            objs = ooi_data[i]['bboxes']
            
            human_boxes = []
            obj_keys = list(objs.keys())
            
            for o in obj_keys:
                temp_cat = objs[o]['class']
                if temp_cat == 'hand':
                    human_boxes.append(int(o))
            
            if len(human_boxes) == 0:
                continue
            
            temp_rels = ooi_data[i]['relations']

            for r in temp_rels:

                obj1, obj2 = r[0]
                
                if hoi_ooi_both == 'hoi':
                    if not((obj1 in human_boxes) or (obj2 in human_boxes)):
                        continue

                if hoi_ooi_both == 'ooi':
                    if (obj1 in human_boxes) or (obj2 in human_boxes):
                        continue

                
                temp_labels = []

                rels = r[1]
                
                for k in selection:
                    temp_labels+=rels[k]

                for t in temp_labels:
                    if t == '':
                        continue
                    temp_col = labels_enumeration[t]
                    X[curr_row, temp_col]+=1
            
            num_gifs_for_each_vid[curr_row] += 1
        
        return [X, Y, num_gifs_for_each_vid]

    from copy import deepcopy as copy

    def normalize(X, Y, count, softmax=1):
        
        offset = 0.00001
        
        X_ret = copy(X)
        
        normalizing_vec = np.sum(X, axis=1) + offset
        normalizing_vec = np.expand_dims(normalizing_vec, 1)
        
        # print(normalizing_vec.shape, X_ret.shape)
        
        X_ret = np.divide(X_ret, normalizing_vec)

        normalizing_vec = count + offset
        normalizing_vec = np.expand_dims(normalizing_vec, 1)

        X_ret = np.divide(X_ret, normalizing_vec)
        
        if softmax == 1:
            import torch
            X_ret = torch.from_numpy(X_ret)
            X_ret_ = torch.nn.functional.softmax(X_ret, dim=1)
            X_ret = X_ret_.numpy()

        return X_ret
    X_hoi, Y, num_gifs_for_each_vid = make_hoi_features(ooi_data,
                                                        videos_enumeration,
                                                        labels_enumeration,
                                                        activity_enumeration,
                                                        vid_to_class, 'hoi')

    X_hoi_normalized = normalize(X_hoi, Y, num_gifs_for_each_vid)

    X_ooi, Y, num_gifs_for_each_vid = make_hoi_features(ooi_data,
                                                        videos_enumeration,
                                                        labels_enumeration,
                                                        activity_enumeration,
                                                        vid_to_class, 'ooi')

    X_ooi_normalized = normalize(X_ooi, Y, num_gifs_for_each_vid)

    X_both, Y, num_gifs_for_each_vid = make_hoi_features(ooi_data,
                                                        videos_enumeration,
                                                        labels_enumeration,
                                                        activity_enumeration,
                                                        vid_to_class, 'both')

    X_both_normalized = normalize(X_both, Y, num_gifs_for_each_vid)
    


    # print(" Concatenation is being DONE")
    X_both2 = np.concatenate((X_hoi, X_ooi), axis=1)
    
    X_both_normalized2 = normalize(X_both2, Y, num_gifs_for_each_vid)
    # X_both_normalized2 = X_both2

    from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
    from sklearn.model_selection import train_test_split # Import train_test_split function
    from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.neural_network import MLPClassifier
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.svm import SVC
    from sklearn.naive_bayes import GaussianNB

    import warnings
    warnings.filterwarnings('ignore')

    def train_test_individual(X_train, Y_train, X_test, Y_test):

        clf = DecisionTreeClassifier()
        # clf = KNeighborsClassifier()
        input_dim = X_train.shape[1]
        # clf = MLPClassifier_pytorch(input_dim=input_dim)
        # clf = KNeighborsClassifier()
        # clf = GaussianNB()

        clf = clf.fit(X_train,Y_train)
        
        Y_pred = clf.predict(X_test)

        # Model Accuracy, how often is the classifier correct?
        
        acc = metrics.accuracy_score(Y_test, Y_pred)
        
        del clf
        del Y_pred
        
        return acc

    def train_test(X_hoi, X_ooi, X_both, X_both2, Y):
        
        import random
        ind = [i for i in range(438)]
        random.shuffle(ind)
        ind = np.array(ind)

        offset = 350
        ind_test = ind[offset:]
        ind_train = ind[:offset]

        X_hoi_train = X_hoi[ind_train, :]
        X_hoi_test = X_hoi[ind_test, :]

        X_ooi_train = X_ooi[ind_train, :]
        X_ooi_test = X_ooi[ind_test, :]

        X_both_train = X_both[ind_train, :]
        X_both_test = X_both[ind_test, :]

        X_both2_train = X_both2[ind_train, :]
        X_both2_test = X_both2[ind_test, :]

        Y_train = Y[ind_train]
        Y_test = Y[ind_test]

        acc_both = train_test_individual(X_both_train, Y_train, X_both_test, Y_test)
        acc_both2 = train_test_individual(X_both2_train, Y_train, X_both2_test, Y_test)
        acc_ooi = train_test_individual(X_ooi_train, Y_train, X_ooi_test, Y_test)
        
        acc_hoi = train_test_individual(X_hoi_train, Y_train, X_hoi_test, Y_test)
        
        Y_random = np.random.randint(0, 12, len(Y_test))
        acc_random = metrics.accuracy_score(Y_test, Y_random)

        # print("Accuracy hoi:", acc_hoi)
        # print("Accuracy ooi:",acc_ooi)
        # print("Accuracy both:",acc_both)
        # print("Accuracy random:",acc_random)
        
        acc = [acc_hoi, acc_ooi, acc_both, acc_both2, acc_random]
        
        return acc

    all_accs = []

    from tqdm import tqdm as tqdm
    for i in tqdm(range(50)):
        temp_acc = train_test(X_hoi_normalized, X_ooi_normalized, X_both_normalized, X_both_normalized2, Y)
        # temp_acc = train_test(X_hoi, X_ooi, X_both, Y)
        all_accs.append(temp_acc)
        
    all_accs = np.array(all_accs)
    all_accs = np.mean(all_accs, axis = 0)
    print(selection)
    print('hoi ooi both both2 random')
    # print(all_accs)
    
    features = {}
    
    features['hoi'] = X_hoi_normalized
    features['ooi'] = X_ooi_normalized
    features['hoi+ooi'] = X_both_normalized
    features['hoi|ooi'] = X_both_normalized2
    features['y'] = Y
    
    return all_accs, features
    
    

In [13]:
import io
import torch
import pickle as pkl

class CPU_Unpickler(pkl.Unpickler):
    def find_class(self, module, name):
        if module == 'torch.storage' and name == '_load_from_bytes':
            return lambda b: torch.load(io.BytesIO(b), map_location='cpu')
        else:
            return super().find_class(module, name)

#contents = pickle.load(f) becomes...
f = open('./labels.pkl', 'rb')
ooi_data = CPU_Unpickler(f).load()
f.close()

selection = ['scr', 'lr', 'mr']

import itertools
selections = []

stuff = selection
for L in range(len(stuff) + 1):
    for subset in itertools.combinations(stuff, L):
        if len(subset) > 0:
            selections.append(list(subset))


In [27]:
# do_everything(selection, ooi_data)

for s in selections:
    accs, feats = do_everything(s, ooi_data)
    print(accs)

100%|██████████| 50/50 [00:01<00:00, 45.74it/s]


['scr']
hoi ooi both both2 random
[0.22727273 0.18409091 0.19227273 0.24159091 0.08318182]


100%|██████████| 50/50 [00:01<00:00, 41.87it/s]


['lr']
hoi ooi both both2 random
[0.20863636 0.3        0.27522727 0.32       0.08272727]


100%|██████████| 50/50 [00:01<00:00, 35.86it/s]


...

['scr', 'mr']
hoi ooi both both2 random
[0.25704545 0.33113636 0.31       0.35840909 0.08454545]


100%|██████████| 50/50 [00:01<00:00, 36.30it/s]


['lr', 'mr']
hoi ooi both both2 random
[0.30159091 0.40795455 0.37863636 0.43159091 0.08159091]


100%|██████████| 50/50 [00:01<00:00, 37.48it/s]

['scr', 'lr', 'mr']
hoi ooi both both2 random
[0.30522727 0.39840909 0.38659091 0.42454545 0.0825    ]


In [18]:
for s in selections:
    accs, feats = do_everything(s, ooi_data)
    print(accs)

100%|██████████| 50/50 [00:00<00:00, 66.61it/s]


['scr']
hoi ooi both both2 random
[0.23636364 0.18590909 0.18636364 0.25818182 0.08318182]


100%|██████████| 50/50 [00:00<00:00, 54.96it/s]


['lr']
hoi ooi both both2 random
[0.20477273 0.29795455 0.26636364 0.29613636 0.08022727]


100%|██████████| 50/50 [00:01<00:00, 47.42it/s]


...

['scr', 'mr']
hoi ooi both both2 random
[0.25545455 0.34204545 0.32068182 0.33772727 0.0875    ]


100%|██████████| 50/50 [00:01<00:00, 46.03it/s]


['lr', 'mr']
hoi ooi both both2 random
[0.275      0.3875     0.37363636 0.38681818 0.07431818]


100%|██████████| 50/50 [00:01<00:00, 46.64it/s]

['scr', 'lr', 'mr']
hoi ooi both both2 random
[0.30681818 0.41136364 0.36659091 0.39727273 0.08340909]


In [21]:
feats.keys()

dict_keys(['hoi', 'ooi', 'hoi+ooi', 'hoi|ooi', 'y'])

In [25]:
feats['hoi|ooi'].shape

(438, 40)